In [ ]:
!pip install numpy tensorflow scikit-learn


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
from PIL import Image


In [ ]:
def load_images_and_masks(img_dir, mask_dir):
    img_files = [os.path.join(img_dir, file) for file in sorted(os.listdir(img_dir))]
    mask_files = [os.path.join(mask_dir, file) for file in sorted(os.listdir(mask_dir))]

    images = [np.array(Image.open(file)) for file in img_files]
    masks = [np.array(Image.open(file)) for file in mask_files]

    return images, masks


In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_data(images, masks):
    images = np.array(images, dtype=np.float32) / 255.0  # Normalize images and use float32
    
    # Convert mask pixel values to categorical labels
    label_map = {255: 3, 170: 2, 85: 1, 0: 0}
    masks = np.array(masks, dtype=np.uint8)
    masks_categorical = np.vectorize(label_map.get)(masks)
    
    return images, masks_categorical


In [ ]:
def split_data(images, masks, train_size=0.8, test_size=0.1):
    # Split into train+val and test
    images_train_val, images_test, masks_train_val, masks_test = train_test_split(
        images, masks, test_size=test_size, random_state=42)

    # Split train into train and val
    val_size = test_size / (1.0 - test_size)  # Adjust validation size
    images_train, images_val, masks_train, masks_val = train_test_split(
        images_train_val, masks_train_val, test_size=val_size, random_state=42)
    
    return images_train, images_val, images_test, masks_train, masks_val, masks_test


In [ ]:
# Paths to your datasets
image_directory = 'N:\\My Drive\\Data\\RUH'
mask_directory = 'N:\\My Drive\\Data\\Mask'

# Load and preprocess data
images, masks = load_images_and_masks(image_directory, mask_directory)
images, masks = preprocess_data(images, masks)

# Split data
images_train, images_val, images_test, masks_train, masks_val, masks_test = split_data(images, masks)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
from tensorflow.keras.layers import BatchNormalization, Dropout

def unet_model(input_size=(256, 256, 1), num_classes=4):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    drop1 = Dropout(0.5)(pool1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(drop1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    drop2 = Dropout(0.5)(pool2)

    # Bridge
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(drop2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)

    # Decoder
    up1 = UpSampling2D(size=(2, 2))(conv3)
    merge1 = concatenate([conv2, up1], axis=3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge1)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    up2 = UpSampling2D(size=(2, 2))(conv4)
    merge2 = concatenate([conv1, up2], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge2)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)

    # Output
    conv6 = Conv2D(num_classes, 1, activation='softmax', padding='same', kernel_initializer='he_normal')(conv5)

    model = Model(inputs=inputs, outputs=conv6)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
model = unet_model(input_size=(256, 256, 1), num_classes=4)
model.summary()


In [ ]:
# Reshape the images and masks to remove the unnecessary dimensions
images_train = images_train.reshape((27647, 256, 256, 3))
masks_train = masks_train.reshape((27647, 256, 256, 1))

# Verify the new shapes
print("New shape of images_train:", images_train.shape)
print("New shape of masks_train:", masks_train.shape)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create a data generator for the training and validation data
def create_datagen():
    return ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')

train_datagen = create_datagen()
val_datagen = create_datagen()

# Prepare the generators
train_generator = train_datagen.flow(images_train, masks_train, batch_size=16)
val_generator = val_datagen.flow(images_val, masks_val, batch_size=16)  # Make sure images_val and masks_val are also correctly shaped

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(images_train) // 16,  # Total number of batches per epoch
    epochs=10,  # Number of epochs to train for
    validation_data=val_generator,
    validation_steps=len(images_val) // 16  # Total number of validation batches
)


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
